In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = 'Newtraining.txt'

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 18706 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

he said the way my blue eyes shined put those georgia stars to shame that night i said  "that's a lie" just a boy in a chevy truck that had a tendency of gettin' stuck on back roads at night and i was right there beside him all summer long and then t


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

31 unique characters


In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ' ' :   0,
  '"' :   1,
  "'" :   2,
  '-' :   3,
  '4' :   4,
  '5' :   5,
  'a' :   6,
  'b' :   7,
  'c' :   8,
  'd' :   9,
  'e' :  10,
  'f' :  11,
  'g' :  12,
  'h' :  13,
  'i' :  14,
  'j' :  15,
  'k' :  16,
  'l' :  17,
  'm' :  18,
  'n' :  19,
  ...
}


In [8]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'he said the w' ---- characters mapped to int ---- > [13 10  0 23  6 14  9  0 24 13 10  0 27]


In [9]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

h
e
 
s
a


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'he said the way my blue eyes shined put those georgia stars to shame that night i said  "that\'s a lie'
'" just a boy in a chevy truck that had a tendency of gettin\' stuck on back roads at night and i was r'
'ight there beside him all summer long and then the time we woke up to find that summer gone  but when'
' you think tim mcgraw i hope you think my favorite song the one we danced to all night long the moon '
'like a spotlight on the lake when you think happiness i hope you think that little black dress think '


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'he said the way my blue eyes shined put those georgia stars to shame that night i said  "that\'s a li'
Target data: 'e said the way my blue eyes shined put those georgia stars to shame that night i said  "that\'s a lie'


In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 13 ('h')
  expected output: 10 ('e')
Step    1
  input: 10 ('e')
  expected output: 0 (' ')
Step    2
  input: 0 (' ')
  expected output: 23 ('s')
Step    3
  input: 23 ('s')
  expected output: 6 ('a')
Step    4
  input: 6 ('a')
  expected output: 14 ('i')


In [14]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [15]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [17]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 31) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           7936      
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 31)            31775     
Total params: 3,978,015
Trainable params: 3,978,015
Non-trainable params: 0
_________________________________________________________________


In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
sampled_indices

array([19, 17,  0, 23, 22, 15,  3, 16, 29, 11, 24,  9, 14, 10,  1,  0,  8,
       28, 15,  1, 10,  0, 13,  4,  8,  8, 28,  4, 30, 30, 10, 13, 10,  1,
       16,  7,  7,  1,  0, 22, 30, 11, 28, 30,  3,  4,  9, 28, 20,  1, 24,
       22, 14, 25, 24,  8,  4,  8,  8, 29, 27, 26, 19,  5,  4, 19, 13, 20,
        6, 20, 27,  1, 24, 15, 23, 26, 27,  6, 17, 29, 17, 14, 27, 16, 10,
       10, 14, 22, 11, 25,  3,  9, 21, 28, 27,  7, 25,  7, 21, 24],
      dtype=int64)

In [22]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ant to and you just see right through me  but if you only knew me  we could be a beautiful miracle  '

Next Char Predictions: 
 'nl srj-kyftdie" cxj"e h4ccx4zzehe"kbb" rzfxz-4dxo"triutc4ccywvn54nhoaow"tjsvwalyliwkeeirfu-dpxwbubpt'


In [23]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 31)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       3.434579


In [24]:
model.compile(optimizer='adam', loss=loss)

In [25]:
# Directory where the checkpoints will be saved
checkpoint_dir = 'taylorswift_training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [26]:
EPOCHS=700

In [27]:
# history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

In [28]:
tf.train.latest_checkpoint(checkpoint_dir)

'taylorswift_training_checkpoints\\ckpt_700'

In [29]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            7936      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 31)             31775     
Total params: 3,978,015
Trainable params: 3,978,015
Non-trainable params: 0
_________________________________________________________________


In [31]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [32]:
print(generate_text(model, start_string=u"something in the way"))

something in the way this year  to save me from tear she's never gonna see the light  no matter what you do  and all i think about is how to make you think stare wouldn't have told nobody if i died  died for you  died for you  oh  what a shame  what a rainy ending given to a perfect day just walk away  no use defending words that you will never say and now that i'm sitting here thinking it through i've never been anywhere cold as you  i didn't know what i want  so don't ask me cause i'm all a part of you and now it's all too late so christmas i gave you my heart but the very next day  you gave it away this year  to save me from tears i'll give it to someone special  a crowded room and friendsomen wouldn't have told nobody if i died  died for you  died for you  oh  what a shame  what a rainy ending given to a perfect day just walk away  no use defending words that you will never say and now that i'm sitting here thinking it through i've never been anywhere cold as you  i didn't know whe

In [34]:
import tkinter as tk
import requests

HEIGHT = 500
WIDTH = 600

def test_function(entry):
	print("This is the entry:", entry)


def get_song(city):

	label.insert(tk.INSERT,generate_text(model, start_string=city))



root = tk.Tk()

root.title("Magenta")

canvas = tk.Canvas(root, height=HEIGHT, width=WIDTH)
canvas.pack()

background_image = tk.PhotoImage(file='landscape3.png')
background_label = tk.Label(root, image=background_image)
background_label.place(relwidth=1, relheight=1)

frame = tk.Frame(root, bg='#80c1ff', bd=5)
frame.place(relx=0.5, rely=0.1, relwidth=0.75, relheight=0.1, anchor='n')

entry = tk.Entry(frame, font=("Helvetica", 15))
entry.place(relwidth=0.65, relheight=1)

button = tk.Button(frame, text="Generate Song", font=40, command=lambda: get_song(entry.get()))
button.place(relx=0.7, relheight=1, relwidth=0.3)

lower_frame = tk.Frame(root, bg='#80c1ff', bd=10)
lower_frame.place(relx=0.5, rely=0.25, relwidth=0.75, relheight=0.6, anchor='n')

label = tk.Text(lower_frame,font=("Helvetica", 15))
label.place(relwidth=1, relheight=1)

root.mainloop()